# 2D Metallic Flat Plate Magnetized in Exterior Field 
# FEM Solution of the Poisson Equation with Discontinuous Coefficient 
# using Ferrite.jl  

## Import Packages

In [1]:
using SparseArrays
using Ferrite

## Section 1: Introduction 

<b>Goal</b>: The goal of this notebook is to provide a reference solution for the magnetization $\vec{M}$ for the 2D flat plate placed inside an exterior magnetic field. 

<b>Questions</b>:
1. dimension plate? 
2. strenght of external field? 
3. resulting values of magnetization components, its direction and norm? 

## Section 2: Test Case Description 

### Geometry

The geometry $\Omega = \Omega_{air} \cup \Omega_{plate}$ consists of the following two elements: 
1. $\Omega_{plate}$: a plate represented as a rectangle of $xxx$ by $yyy$ meters along the $x$ and $y$ axis. Origin in the middle of the plate; 
2. $\Omega_{air}$: a surrounding box of air as a rectangle $xxx$ by $yyy$ meters along the $x$ and $y$ axix; 

Remarks: 
1. Boundaries of the air box represent the far-field. Not sure whether box of air is large enough; 
2. Possibly take advance of symmetry in the field to reduce the computational domain to first octant of the coordinate axes. Use symmetry boundary conditions;

### Mesh 

Generate a uniform mesh using Ferrite.jl build in mesh generator. 

Remarks
1. Mesh allowed to be coarser in air than in plate domain. This, however, requires resorting to GMSH as mesh generator; 

### Physics 

<b>Air</b> has exterior (unperturbed) magnetic field in $x$-direction given by $\vec{H}_0(\vec{x}) = (H_0,0,0)$ (units A/m). The scalar magnetic potential $V_m(\vec{x})$ for this field is given by $V_m(\vec{x}) = H_0 \, x$ (units A). Indeed, we have that by definition of the potential that $\vec{H}_0(\vec{x}) = \text{grad} \, V_m(\vec{x}) = \nabla V_m(\vec{x}) = (H_0,0,0)$. The potential $V_m(\vec{x}) = H_0 \, x$ will be applied as Dirichlet (fixedValue) boundary condition for the potential.   

<b>Magnetic plate</b> has magnetic susceptibility $\chi_m$, e.g., $\chi_m = 100$ (dimensionless). Plate placed in magnetic field will deform field locally (higher $\chi_m$ implies larger deformation of the exterior field due to metallic plate).  

The governing equations for stationary magnetic field are: 

$ \nabla \times \vec{H}(\vec{x}) = \vec{0}$ 

$ \nabla \cdot \vec{B}(\vec{x}) = 0 $

$\vec{B}(\vec{x}) = \mu_0 \, \vec{H}(\vec{x})$ (in air) 
and $\vec{B}(\vec{x}) = \mu_0 \, (1 + \chi_m) \, \vec{H}(\vec{x})$

<b>Differential Equation</b> 

The Poisson equation for the scalar magnetic potential $V_m(\vec{x})$ such that $\vec{H}(\vec{x}) = \nabla V_m(\vec{x})$ is given by 

$$
\nabla \cdot \left[ \mu_0 \, (1 + \chi_m) \, V_m(\vec{x}) \right] = 0 \text{ for } \vec{x} \in \Omega = \Omega_{air} \cup \Omega_{plate} 
$$

where $\chi_m = 0$ for $\vec{x} \in \Omega_{air}$ and $\chi_m = 100$ (dimensionless) for $\vec{x} \in \Omega_{plate}$.

<b>Boundary Conditions</b> 

As boundary condition on the scalar field $V_m(\vec{x})$ we impose that 
 
$$ V_m(\vec{x}) = H_0 \, x  \text{ for } \vec{x} \in \partial \Omega $$. 

### Solver

Using [Ferrite](https://ferrite-fem.github.io/Ferrite.jl/stable/examples/heat_equation/). Possibly consider [Gridap](https://gridap.github.io/Tutorials/dev/pages/t001_poisson/) as alternative in a later stage should the neeed arrise. 

### Post-Processing 

Compute the following quantities : 
1. the magnetic field $\vec{H} = \text{grad} [ u(\vec{x}) ]$ (the $x$, $y$, $z$ component and the magnitude) in the plate and air domain;
2. the magnetic flux $\vec{B} = \mu_0 \, \vec{H}$ (in air) and $\vec{B} = \mu_0 \, (1 + \chi_m) \, \vec{H}$; (as before)
3. the magnetization vector $\vec{M} = \chi_m \, \vec{H}$ in plate; (as before)

## Section 3: Ferrite.jl Tiny Manual  

### Ferrite.jl Howto: Generate Grid for a rectangle in 1, 2 or 3 dimensions  

In [42]:
?generate_grid

search: generate_grid



```
generate_grid(celltype::Cell, nel::NTuple, [left::Vec, right::Vec)
```

Return a `Grid` for a rectangle in 1, 2 or 3 dimensions. `celltype` defined the type of cells, e.g. `Triangle` or `Hexahedron`. `nel` is a tuple of the number of elements in each direction. `left` and `right` are optional endpoints of the domain. Defaults to -1 and 1 in all directions.


In [43]:
# generate tiny grid for testing purposes 
nels  = (2, 2) # number of elements in each spatial direction
left  = Vec((0.0, 0.0)) # start point for geometry 
right = Vec((1.0, 1.0,))    # end point for geometry
grid = generate_grid(Quadrilateral, nels,left, right) 

Grid{2, Quadrilateral, Float64} with 4 Quadrilateral cells and 9 nodes

In [44]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip)

CellScalarValues{2, Float64, RefCube} with 4 shape functions and 4 quadrature points

### Ferrite.jl Howto: What does re-init do? 

In [45]:
?reinit!

search: reinit! atreplinit parentindices CartesianIndex CartesianIndices



```
reinit!(cv::CellValues, x::Vector)
reinit!(bv::FaceValues, x::Vector, face::Int)
```

Update the `CellValues`/`FaceValues` object for a cell or face with coordinates `x`. The derivatives of the shape functions, and the new integration weights are computed.


### Ferrite.jl Howto:  How to get coords and coord_qp?
Obtain coordinates of quadrature points in each cell by 
1. loop over cells in the mesh using CellIterator(dh), where dh = DofHandler(grid);
2. for each cell, retrieve the coordinates for each cell using getcoordinates() yielding coord as output; 
3. reinit cellvalues for each cell using reinit!;   
4. loop over quad points of cell and retrieve spatial coordinate for each quad point in the cell using spatial_coordinate() and coords as input; 

In [46]:
#?add!

In [47]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [48]:
function my_diff_coeff(coord_qp)
    return coord_qp[1] + coord_qp[2]
end 

my_diff_coeff (generic function with 1 method)

In [49]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        # display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords(dh) 

0

In [50]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        # display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords (generic function with 1 method)

In [51]:
my_coords = my_get_coords(dh)
my_coords

0

## Section 4: 2D Mockup of the Magnetized Plate  

In [112]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(coord_qp)
    xbound = abs(coord_qp[1])<0.3
    ybound = abs(coord_qp[2])<0.1    
    return   
end 

# Ke: added spatially varying diffusion coefficient 
# fe: forces zero source term 
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues, mycoords)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # ADDED: Get coord of quadrature point
        coords_qp = spatial_coordinate(cellvalues, q_point, mycoords)
        # ADDED: Evaluate spatially dependent diffusion coefficient in quad point 
        val_diff_coeff = my_diff_coeff(coords_qp)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += 0 * δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # MODIFIED: Add contribution to Ke
                Ke[i, j] += val_diff_coeff * (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

assemble_element! (generic function with 1 method)

In [113]:
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Added: Get coordinates from current cell 
        coords = getcoordinates(cell)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Modified - Compute element contribution
        assemble_element!(Ke, fe, cellvalues, coords)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [114]:
nels  = (50, 50) # number of elements in each spatial direction
left  = Vec((-1.0, -1.0)) # start point for geometry 
right = Vec((1.0, 1.0,))    # end point for geometry
grid = generate_grid(Quadrilateral,nels,left,right);

# nels  = (100, 50) # number of elements in each spatial direction
# left  = Vec((0.0, 0.0)) # start point for geometry 
# right = Vec((2.0, 1.0,))    # end point for geometry
# grid = generate_grid(Quadrilateral,nels,left,right);
#grid 

In [115]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip);

In [116]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [117]:
K = create_sparsity_pattern(dh);

In [118]:
ch = ConstraintHandler(dh);

In [119]:
∂Ω = union(
    getfaceset(grid, "left"),
    getfaceset(grid, "right"),
    getfaceset(grid, "top"),
    getfaceset(grid, "bottom"),
);

In [120]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> x[1])
add!(ch, dbc);

In [121]:
close!(ch)

ConstraintHandler:
  BCs:
    Field: u, Components: [1]

In [122]:
K, f = assemble_global(cellvalues, K, dh);

LoadError: MethodError: no method matching *(::Nothing, ::Float64)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:385
[0m  *([91m::StridedArray{P}[39m, ::Real) where P<:Dates.Period at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:44
[0m  ...

In [123]:
apply!(K, f, ch)
u = K \ f;

LoadError: LinearAlgebra.SingularException(1)

In [124]:
vtk_grid("magnetic_plate", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

1-element Vector{String}:
 "magnetic_plate.vtu"

## Section 5: Computing Fluxes

In [100]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords (generic function with 1 method)

In [104]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(coord_qp)
    xbound = abs(coord_qp[1])<0.3
    ybound = abs(coord_qp[2])<0.2 
    return 4*pi*1e-6*(1+99*(xbound*ybound)) 
end 

my_diff_coeff (generic function with 1 method)

In [105]:
function compute_hfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)

        for q_point in 1:nqp
            q_qp = - function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

function compute_bfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        
        for q_point in 1:nqp
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            val_diff_coeff = my_diff_coeff(coords_qp)
            q_qp = - val_diff_coeff*function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

compute_bfield (generic function with 1 method)

In [106]:
h_gp = compute_hfield(cellvalues, dh, u);
b_gp = compute_bfield(cellvalues, dh, u);
#q_gp

In [107]:
projector = L2Projector(ip, grid);
#projector 

In [108]:
#?project

In [109]:
h_projected = project(projector, h_gp, qr; project_to_nodes=false);
b_projected = project(projector, b_gp, qr; project_to_nodes=false);
# q_projected

## Section 6: Exporting to VTK 

In [110]:
#?vtk_point_data

In [111]:
vtk_grid("2d_uniform_magn_plate", dh) do vtk
    vtk_point_data(vtk, dh, u, "Vm")
    vtk_point_data(vtk, projector, h_projected, "H")
    vtk_point_data(vtk, projector, b_projected, "B")
end

1-element Vector{String}:
 "2d_uniform_magn_plate.vtu"